In [135]:
import pandas as pd
train_data=pd.read_csv('data/train.csv')
#test_data=pd.read_csv('data/test.csv')
print(train_data.head())
#print(test_data.head())

   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  


In [136]:
train_data=train_data.drop(columns=['keyword','location'])
#test_data=test_data.drop(columns=['keyword','location'])
print(train_data.head())
#print(test_data.head())

   id                                               text  target
0   1  Our Deeds are the Reason of this #earthquake M...       1
1   4             Forest fire near La Ronge Sask. Canada       1
2   5  All residents asked to 'shelter in place' are ...       1
3   6  13,000 people receive #wildfires evacuation or...       1
4   7  Just got sent this photo from Ruby #Alaska as ...       1


In [137]:
X=train_data.drop('target',axis=1)
y=train_data['target']
print(X.shape)
print(y.shape)

(7613, 2)
(7613,)


In [138]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [180]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional

In [141]:
voc_size=8000
tweets=X.copy()

In [142]:
#one_hot representation
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [143]:
from nltk.stem import WordNetLemmatizer
wordnet=WordNetLemmatizer()
corpus_lemmatization=[]

for i in range(len(tweets)):
    review=re.sub('[^a-zA-Z]',' ',tweets['text'][i])
    review=review.lower()
    review=review.split()

    review=[wordnet.lemmatize(word) for word in review if not word in set (stopwords.words('english'))]
    review=' '.join(review)
    corpus_lemmatization.append(review)

In [144]:
onehot_representation=[one_hot(words,voc_size)for words in corpus_lemmatization]
onehot_representation

[[5771, 2581, 2409, 2317, 620, 1889, 5671],
 [1350, 6784, 6232, 102, 681, 3968, 2684],
 [2325, 1440, 2585, 4996, 6467, 4827, 815, 2585, 4996, 2515, 720],
 [3057, 2641, 7402, 815, 2515, 6385],
 [72, 6306, 6574, 1331, 4063, 2513, 7402, 1031, 6931],
 [7042, 924, 6385, 3645, 5104, 6972, 4928, 4036, 5328, 6784, 2955, 7402],
 [5000, 1845, 6585, 7578, 4990, 7504, 5706, 5109, 1128, 7611, 1709, 3256],
 [885, 6106, 6145, 6784, 1795],
 [414, 815, 6861, 5484, 5585, 5109],
 [7048, 6177, 4901, 3256],
 [2666, 3057, 3304, 2214, 2092, 4339],
 [35,
  1912,
  3663,
  4434,
  802,
  5491,
  760,
  2012,
  2719,
  1912,
  3663,
  6756,
  6756,
  946,
  5706],
 [1854, 5706, 3176, 7209, 3663, 1124, 2692, 5574],
 [5000, 3504, 3432, 3173, 3504],
 [6477, 6931, 2512, 6322, 3691, 1155, 1415],
 [7247],
 [6689, 1396],
 [4584, 4701],
 [3691, 2796],
 [4909],
 [7543],
 [6189, 402],
 [6689, 1039],
 [6842, 1124],
 [3823],
 [4765, 6544, 5178],
 [3574, 4116, 4296],
 [6689, 6895],
 [35],
 [3013, 7618],
 [2573],
 [7966, 491

In [145]:
embedded_docs=pad_sequences(onehot_representation,padding='pre',maxlen=20)
embedded_docs

array([[   0,    0,    0, ...,  620, 1889, 5671],
       [   0,    0,    0, ...,  681, 3968, 2684],
       [   0,    0,    0, ..., 4996, 2515,  720],
       ...,
       [   0,    0,    0, ...,  306, 7794,  629],
       [   0,    0,    0, ...,  355, 3030, 5244],
       [   0,    0,    0, ...,  306, 7303, 1474]])

In [267]:
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=20))
#model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
#model.add(Dropout(0.3))
model.add(Dense(2,activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [268]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 20, 100)           800000    
                                                                 
 bidirectional_9 (Bidirecti  (None, 200)               160800    
 onal)                                                           
                                                                 
 dense_20 (Dense)            (None, 2)                 402       
                                                                 
Total params: 961202 (3.67 MB)
Trainable params: 961202 (3.67 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [269]:
len(embedded_docs),y.shape

(7613, (7613,))

In [270]:
import numpy as np
X_new=np.array(embedded_docs)
y_new=np.array(y)

In [271]:
X_new.shape,y_new.shape

((7613, 20), (7613,))

In [272]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_new,y_new,test_size=0.2,random_state=42)


In [273]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
print(y_train)
y_test = to_categorical(y_test)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [274]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
96/96 [==============================] - 15s 72ms/step - loss: 0.5766 - accuracy: 0.6837 - val_loss: 0.4633 - val_accuracy: 0.7886
Epoch 2/10
96/96 [==============================] - 5s 53ms/step - loss: 0.3391 - accuracy: 0.8591 - val_loss: 0.5065 - val_accuracy: 0.7649
Epoch 3/10
96/96 [==============================] - 5s 48ms/step - loss: 0.2095 - accuracy: 0.9251 - val_loss: 0.6052 - val_accuracy: 0.7557
Epoch 4/10
96/96 [==============================] - 5s 48ms/step - loss: 0.1295 - accuracy: 0.9571 - val_loss: 0.7454 - val_accuracy: 0.7420
Epoch 5/10
96/96 [==============================] - 5s 49ms/step - loss: 0.0844 - accuracy: 0.9741 - val_loss: 0.8608 - val_accuracy: 0.7387
Epoch 6/10
96/96 [==============================] - 5s 49ms/step - loss: 0.0542 - accuracy: 0.9839 - val_loss: 1.1119 - val_accuracy: 0.7078
Epoch 7/10
96/96 [==============================] - 5s 51ms/step - loss: 0.0436 - accuracy: 0.9877 - val_loss: 1.1998 - val_accuracy: 0.7242
Epoch 8/10
9

In [275]:
y_pred=model.predict(X_test)
print(y_pred[0])
#classes=np.argmax(y_pred[0])
#print(classes)
#type(y_pred[0])

48/48 [==============================] - 2s 8ms/step
[9.9999416e-01 5.8486135e-06]


In [276]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

48/48 [==============================] - 1s 10ms/step
